In [2]:
import numpy as np
import cvxopt
import matplotlib.pyplot as plt
from matplotlib import pylab
from matplotlib import colors
from sklearn.svm import SVC

In [3]:
X = np.array([[1,1],[2,3],[2.2,3],[3,1]])
y = np.array([1.,1.,-1.,-1.])

## no kernel

In [4]:
# no kernel
# use cvxopt find the alphas.
ni, nf = X.shape
X_ = np.matrix(X)
P = cvxopt.matrix(np.outer(y,y) * np.array(X_* X_.T))
q = cvxopt.matrix(-1 * np.ones(ni))
G = cvxopt.matrix(np.diag(-1*np.ones(ni)))
h = cvxopt.matrix(np.zeros(ni))
A = cvxopt.matrix(y,(1,ni))
b = cvxopt.matrix(0.0)

In [5]:
sol = cvxopt.solvers.qp(P,q,G,h,A,b)

     pcost       dcost       gap    pres   dres
 0: -2.1330e+00 -4.9051e+00  3e+00  6e-17  2e+00
 1: -4.4482e+00 -4.9674e+00  5e-01  4e-16  9e-01
 2: -2.8979e+01 -3.7154e+01  8e+00  5e-15  9e-01
 3: -4.9519e+01 -5.9459e+01  1e+01  8e-15  2e-01
 4: -5.0000e+01 -5.0154e+01  2e-01  2e-14  3e-03
 5: -5.0000e+01 -5.0002e+01  2e-03  2e-14  3e-05
 6: -5.0000e+01 -5.0000e+01  2e-05  2e-14  3e-07
 7: -5.0000e+01 -5.0000e+01  2e-07  7e-15  3e-09
Optimal solution found.


In [6]:
alphas = np.ravel(sol['x'])

In [7]:
alphas

array([  4.46345312e-12,   4.99999999e+01,   4.99999999e+01,
         2.85470518e-12])

In [8]:
sv_inds = np.arange(len(y))[alphas>1e-5]

according to alphas, calculate w and b

In [9]:
w = 0
for i in sv_inds:
    w += alphas[i]* X[i] *y[i]

In [10]:
w

array([ -9.99999998e+00,  -4.86011231e-12])

In [11]:
b = 0
for i in sv_inds:
    temp = np.dot(w,X[i].T)
    b += (y[i] - temp)
b /= len(sv_inds)

In [12]:
b

20.999999961940965

## kernels

In [13]:
#kernels function
def linear_kernel(x1,x2):
    return np.dot(x1,x2)

In [14]:
def poly_kernel(x1,x2,d=2):
    return (np.dot(x1,x2) +1) **d

In [15]:
def gau(x1,x2,s):
    return np.exp(-1*(np.dot(x1-x2,x1-x2)/(2*(s**2))))

In [16]:
def tanh_kernel(x1,x2):
    return np.tanh(2*np.dot(x1,x2)+1)

In [17]:
#define which kernel function we want to use.
#kf = lambda x1,x2: radial_basis_kernel(x1,x2,3)
kf = linear_kernel

In [18]:
def kernel_matrix(X,kf):
    ns, nf = X.shape
    km = np.zeros((ns,ns))
    for i in range(ns):
        for j in range(ns):
            km[i,j] = kf(X[i],X[j])
    return km

In [19]:
def classify(X,y,alphas,b,kf,x):
    sv_inds = np.arange(len(y))[alphas > 1e-5]
    p=0
    for i in sv_inds:
        p += alphas[i]*y[i]*kf(X[i],x)
    return p+b

In [20]:
def compute_b(X,y,alphas,kf):
    sv_inds = np.arange(len(y))[alphas > 1e-5]
    b = 0
    print(sv_inds)
    for i in sv_inds:
        b += (y[i] - classify(X,y,alphas,0,kf,X[i]))
    return b/len(sv_inds)

In [23]:
#use cvxopt to calculate alphas
ni, nf = X.shape
P = cvxopt.matrix(np.outer(y,y) * kernel_matrix(X,kf))
q = cvxopt.matrix(-1 * np.ones(ni))
G = cvxopt.matrix(np.diag(-1*np.ones(ni)))
h = cvxopt.matrix(np.zeros(ni))
A = cvxopt.matrix(y,(1,ni))
b = cvxopt.matrix(0.0)

In [24]:
sol = cvxopt.solvers.qp(P,q,G,h,A,b)

     pcost       dcost       gap    pres   dres
 0: -1.9296e+00 -4.4806e+00  3e+00  2e-16  2e+00
 1: -3.7977e+00 -4.2374e+00  4e-01  4e-16  8e-01
 2:  1.6094e+01 -8.0054e+01  1e+02  3e-14  3e-14
 3: -1.4330e+01 -2.5127e+01  1e+01  1e-14  4e-14
 4: -1.7858e+01 -1.8069e+01  2e-01  8e-15  4e-14
 5: -1.7999e+01 -1.8001e+01  2e-03  2e-15  1e-14
 6: -1.8000e+01 -1.8000e+01  2e-05  8e-15  3e-14
 7: -1.8000e+01 -1.8000e+01  2e-07  4e-15  1e-14
Optimal solution found.


In [25]:
alphas = np.ravel(sol['x'])

In [26]:
# calculate function margin(use gaussian kernel), used in quize 11. 
# new is the new x to be classified. function margin = sum(alphas[i] * y[i] * gaussian(x[i],new x))
new = [1,4]
sv = [1,2]
func_m = 0
for i in sv:
    func_m += 10 * y[i] * gau(X[i],new,1)
func_m

1.1852723239414615